# Proyecto de Web Scrapping del Marketplace Yapo.

1. Scrapping de Datos - Yapo

In [3]:
#Importamos las librerias necesarias
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver import ChromeOptions
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
import scipy as sp
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

#Automcompletar rápido
%config IPCompleter.greedy=True

#Display 6 registros
pd.options.display.min_rows = 6

Con la libreria bs4, abriremos un navegador Firefox, en la pagina de los anuncios de Yapo

In [6]:
browser = webdriver.Firefox() #Cargamos el driver a una variable
myUrl = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=samsung&pagina="
browser.get(myUrl) #Abrimos Firefox y entramos a la pagina sin loguearnos.
pageSoup = soup(browser.page_source, "html.parser")

In [10]:
def webscraping(numero):
    pagina = "https://www.yapo.cl/region-metropolitana/marketplace/computadores-electronica/celulares?orden=date-desc&buscar=samsung&pagina="
    pag = pagina+str(numero)
    browser.get(pag) #Abrimos Chrome y entramos a la pagina sin loguearnos.
    time.sleep(3)
    pageSoup = soup(browser.page_source, "html.parser")
    
    #Informacion
    #titulo    
    titulo_h2 = pageSoup.find_all("h2", class_="title m-0 text-truncate pb-2")

    #precio
    precios_span = pageSoup.find_all("span", class_="clp-price d-flex display-price ng-star-inserted")

        #fecha de publicacion
    fecha_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #hora de publicacion
    hora_p = pageSoup.find_all("p", class_="d-flex m-0 created col ng-star-inserted")

    #lugar
    lugar_span = pageSoup.find_all("span", class_="label align-self-center")

    #dataframes
    df_2 = pd.DataFrame(0, index=range(50), columns=["pagina", "titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])
    #Reiniciamos el index para quedar con rangeindex y no con un multiindex
    df_2.reset_index(drop=True, inplace=True)


    #Poblamos titulo.
    df_2["titulo"][:len(titulo_h2)] = titulo_h2
    df_2["titulo"] = df_2["titulo"].astype(str)
    df_2["titulo"]= df_2["titulo"].str.replace('<h2 _ngcontent-serverapp-c2620551781="" class="title m-0 text-truncate pb-2">', '').str.replace('</h2>', '')
    df_2["titulo"]= df_2["titulo"].str.replace('!', '').str.replace('¡', '')

    #Poblamos precio
    df_2["precio"][:len(precios_span)] = precios_span
    df_2["precio"] = df_2["precio"].astype(str)
    df_2["precio"] = (df_2["precio"].str.replace(
            '<span _ngcontent-serverapp-c2620551781="" class="clp-price d-flex display-price ng-star-inserted"><span _ngcontent-serverapp-c2620551781="" class="bracket">(</span>$', '').str.replace(
                '<span _ngcontent-serverapp-c2620551781="" class="bracket">)</span></span>', '').str.replace('.',''))

    #Poblamos la fecha
    df_2["fecha_publicacion"][:len(fecha_p)] = fecha_p
    df_2["fecha_publicacion"]= df_2["fecha_publicacion"].astype(str)
    df_2["fecha_publicacion"] = df_2["fecha_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')

    #iteramos para extraer solo la fecha.
    for indice, elemento in enumerate(df_2["fecha_publicacion"]):
        df_2.loc[indice, "fecha_publicacion"] = df_2.loc[indice, "fecha_publicacion"][:10]


    #Poblamos la hora
    df_2["hora_publicacion"][:len(hora_p)] = hora_p
    df_2["hora_publicacion"]= df_2["hora_publicacion"].astype(str)
    df_2["hora_publicacion"] = df_2["hora_publicacion"].str.replace('<p _ngcontent-serverapp-c2620551781="" class="d-flex m-0 created col ng-star-inserted" title="', '')


    #iteramos para extraer solo la hora.
    for indice, elemento in enumerate(df_2["hora_publicacion"]):
        df_2.loc[indice, "hora_publicacion"] = df_2.loc[indice, "hora_publicacion"][12:20]


    #Poblamos el lugar desde donde se publica el producto
    df_2["lugar_venta"][:len(lugar_span)] = lugar_span
    df_2["lugar_venta"] = df_2["lugar_venta"].astype(str)
    df_2["lugar_venta"] = df_2["lugar_venta"].str.replace('<span _ngcontent-serverapp-c2620551781="" class="label align-self-center">', '').str.replace(" <!-- --><!-- -->", "").str.replace("</span>", "")
    
    df_2["pagina"] = numero
    df_2.to_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\01_Samsung\WS Datasets_Samsung\ws_samsung_pagina_{numero}.csv", index=False)


In [11]:
for i in range(1,72):
    if i == 71:
        webscraping(i)
        print(f"Proceso terminado: Ultima pagina {i}")
        break
    else:
        webscraping(i)

C:\Users\miche\AppData\Local\Temp\ipykernel_7748\315940880.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["precio"][:len(precios_span)] = precios_span
c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:976: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
C:\Users\miche\AppData\Local\Temp\ipykernel_7748\315940880.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

Proceso terminado: Ultima pagina 71


C:\Users\miche\AppData\Local\Temp\ipykernel_7748\315940880.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2["precio"][:len(precios_span)] = precios_span
c:\Users\miche\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\dtypes\cast.py:976: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)
C:\Users\miche\AppData\Local\Temp\ipykernel_7748\315940880.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gu

In [4]:
#Creamos un nuevo DataFrame
df_final = pd.DataFrame(0, index=range(1), columns=["pagina","titulo","precio","fecha_publicacion", "hora_publicacion", "lugar_venta"])

#Poblamos el nuevo DataFrame
for i in range(1,72):
    df_nuevo = pd.read_csv(fr"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\01_Samsung\WS Datasets_Samsung\ws_samsung_pagina_{i}.csv")
    df_final = pd.concat([df_final ,df_nuevo],ignore_index=True)
      

In [5]:
#Con Nulos y duplicados - 43901 datos
df_final

,pagina,titulo,precio,fecha_publicacion,hora_publicacion,lugar_venta
0,0,0,0,0,0,0
1,1,Celular Samsung Galaxy S23 + 256 GB,650000,24-08-2023,17:22:23,"Macul, Región Metropolitana"
2,1,Samsung S21 fe,350000,24-08-2023,16:29:47,"Santiago, Región Metropolitana"
...,...,...,...,...,...,...
3548,71,0,0,0,NaN,0
3549,71,0,0,0,NaN,0
3550,71,0,0,0,NaN,0


In [6]:
#Borramos los nulos
df_final.dropna(inplace=True)

In [7]:
#Borramos todas las columnas que tenga 0 en precio
df_final = df_final[df_final["titulo"] != 0]
df_final = df_final[df_final["precio"] != 0]

In [8]:
#Reiniciamos el Dataframe
df_final.reset_index(drop=True, inplace=True)

In [10]:
#sin nulos, duplicados o titulo = 0 - 41244 datos
df_final["titulo"] = df_final["titulo"].str.title()

In [11]:
df_final.drop(columns=["pagina"], inplace=True)

In [24]:
#Mostramos el Dataframe final
df_final.to_csv(r"C:\Users\miche\OneDrive\Escritorio\Web Scrapping\01_Samsung\df_samsung.csv", index=False)